In [4]:
import numpy as np

def scr(valoriscr, matricecorr):
    """
    calcola il Solvency Capital Requirement (SCR) aggregato.
    
    :param valoriscr: Array dei singoli SCR.
    :param matricecorr: Matrice di correlazione tra i rischi.
    :return: Valore aggregato dello SCR.
    """
    valoriscr = np.array(valoriscr)
    matricecorr = np.array(matricecorr)
    
    # verifica che le dimensioni siano compatibili
    if valoriscr.shape[0] != matricecorr.shape[0] or matricecorr.shape[0] != matricecorr.shape[1]:
        raise ValueError("Le dimensioni della matrice di correlazione e del vettore SCR non corrispondono.")
    
    scr_agg = np.sqrt(valoriscr.T @ matricecorr @ valoriscr)
    return scr_agg



# shock da Solvency II
shock_az = 0.39
shock_tasso = 0.015
shock_immob = 0.25
shock_valuta = 0.25
shock_spread = 0.084
soglia_concentr = 50
tasso_concentr = 0.10
shock_vita = 0.15  
shock_non_vita = 0.20  
shock_salute = 0.18

# matrice di correlazione per il rischio di mercato
corr_matrice_mkt = np.array([
    [1.00, 0.50, 0.25, 0.25, 0.50, 0.50],
    [0.50, 1.00, 0.25, 0.25, 0.50, 0.50],
    [0.25, 0.25, 1.00, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 1.00, 0.25, 0.25],
    [0.50, 0.50, 0.25, 0.25, 1.00, 0.50],
    [0.50, 0.50, 0.25, 0.25, 0.50, 1.00]
])

# matrice di correlazione per i rischi assicurativi
corr_matrice_ass = np.array([
    [1.00, 0.25, 0.25],
    [0.25, 1.00, 0.50],
    [0.25, 0.50, 1.00]
])

# matrice di correlazione per l'aggregazione
corr_matrice_finale = np.array([
    [1.00, 0.25, 0.25],
    [0.25, 1.00, 0.50],
    [0.25, 0.50, 1.00]
])

# definizione delle esposizioni (in milioni di euro)
espo_az = 100
espo_obblig = 200
espo_immobil = 50
espo_valuta = 30
espo_spread = 100
espo_concentr = 80
EAD = [50, 30, 40]
PD = [0.02, 0.03, 0.015]  
LGD = [0.6, 0.5, 0.7] 

# esposizioni per rischi assicurativi
espo_vita = 500  
espo_non_vita = 300  
espo_salute = 200  

# calcolo singoli SCR per il rischio di mercato
scr_tasso = espo_obblig * shock_tasso * 8
scr_az = espo_az * shock_az
scr_immobil = espo_immobil * shock_immob
scr_valuta = espo_valuta * (1 - 1 / (1 + shock_valuta))
scr_spread = espo_spread * shock_spread
scr_concentr = max(0, (espo_concentr - soglia_concentr) * tasso_concentr)

# calcolo SCR aggregato per il rischio di mercato
scr_mercato = scr([scr_tasso, scr_az, scr_immobil, scr_valuta, scr_spread, scr_concentr], corr_matrice_mkt)

# calcolo SCR per rischio di controparte
scr_controparti_finale = sum(e * p * l for e, p, l in zip(EAD, PD, LGD))

# calcolo SCR per rischi assicurativi
scr_vita = espo_vita * shock_vita
scr_non_vita = espo_non_vita * shock_non_vita
scr_salute = espo_salute * shock_salute
scr_assic = scr([scr_vita, scr_non_vita, scr_salute], corr_matrice_ass)

# aggregazione finale dello SCR
scr_totale = scr([scr_mercato, scr_controparti_finale, scr_assic], corr_matrice_finale)

# risultati
print(f"\nSCR Totale Rischio di Mercato: {scr_mercato:.2f}M")
print(f"SCR Totale Rischio di Controparte: {scr_controparti_finale:.2f}M")
print(f"SCR Totale Rischi Assicurativi: {scr_assic:.2f}M")
print(f"SCR Totale Aggregato: {scr_totale:.2f}M")


SCR Totale Rischio di Mercato: 69.09M
SCR Totale Rischio di Controparte: 1.47M
SCR Totale Rischi Assicurativi: 127.60M
SCR Totale Aggregato: 160.32M
